In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[7],5
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[3]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -3.062457e-06                    1  4.742729e-06
0.000750        2 -3.074693e-06                    2  1.635975e-05
0.001052        3 -3.096656e-06                    3 -1.797097e-06
0.001476        4 -3.117577e-06                    4 -1.577485e-06
0.002070        5 -3.136871e-06                    5 -2.295197e-06
0.002904        6 -3.154710e-06                    6 -1.855495e-06
0.004074        7 -3.170994e-06                    7 -2.191160e-06
0.005714        8 -3.184315e-06                    8 -2.252414e-06
0.008015        9 -3.195763e-06                    9 -2.176884e-06
0.011243       10 -3.205389e-06                   10 -2.312172e-06
0.015771       11 -3.212033e-06                   11 -2.260261e-06
0.022122       12 -3.216225e-06                   12 -2.361589e-06
0.031031       13 -3.217972e-06                   13 -2.044437e-06
0.043528       14 -3.215579e-06                   14  1.268135e-07
0.061057       15 -3.164833e-06                   15  5.047717e-07
0.085645       16 -2.811961e-06                   16 -1.056959e-06
0.120136       17 -1.774018e-06                   17 -1.532927e-06
0.168516       18  3.174695e-07                   18 -9.854671e-07
0.236378       19  4.012051e-06                   19  2.361737e-07
0.331549       20  1.008080e-05                   20  2.395790e-06
0.465100       21  1.958012e-05                   21  5.758262e-06
0.652400       22  3.229261e-05                   22  1.010938e-05
0.915100       23  3.763563e-05                   23  1.257187e-05
1.283650       24  3.244320e-05                   24  1.098074e-05
1.800600       25  2.441987e-05                   25  7.833618e-06
2.525700       26  1.578892e-05                   26  4.471434e-06
3.542800       27  9.067818e-06                   27  1.829103e-06
4.969550       28  4.365580e-06                   28 -7.282412e-09
6.970850       29  1.158907e-06                   29 -1.233913e-06
9.778100       30 -9.564875e-07                   30 -2.017156e-06
13.715850      31 -2.266144e-06                   31 -2.477234e-06
19.239350      32 -3.003623e-06                   32 -2.710937e-06
26.987250      33 -3.342380e-06                   33 -2.776489e-06
37.855300      34 -3.421479e-06                   34 -2.707207e-06
53.100050      35 -3.231183e-06                   35 -2.577447e-06
73.887500      36 -2.770823e-06                   36 -2.360563e-06
97.662500      37 -2.478024e-06                   37 -2.215577e-06
121.437500     38 -2.769983e-06                   38 -2.490206e-06
145.212500     39 -3.486453e-06                   39 -3.079793e-06
168.987500     40 -4.530767e-06                   40 -3.976670e-06
192.762500     41 -7.396834e-06                   41 -6.673286e-06
216.537500     42 -1.759436e-05                   42 -1.692337e-05
240.312500     43 -3.432805e-05                   43 -3.563992e-05
264.087500     44 -5.578136e-05                   44 -6.387505e-05
287.862500     45 -7.422477e-05                   45 -9.906865e-05
311.637500     46 -7.366136e-05                   46 -1.321618e-04
335.412500     47 -2.859422e-05                   47 -1.436463e-04
359.187500     48  9.427436e-05                   48 -1.044912e-04
382.962500     49  3.155546e-04                   49  2.133819e-05
406.737500     50  6.666359e-04                   50  2.580360e-04
430.512500     51  1.181950e-03                   51  6.401011e-04
454.287500     52  1.897115e-03                   52  1.211716e-03
478.062500     53  2.849152e-03                   53  2.020953e-03
501.837500     54  4.069813e-03                   54  3.100906e-03
525.612500     55  5.603002e-03                   55  4.453908e-03
549.387500     56  7.479904e-03                   56  6.127604e-03
573.162500     57  9.739176e-03                   57  8.15735

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -33.973919  0.000000e+00 -33.973919 -34.013624   
0.000624    2     -33.973919  8.282641e-11 -33.973919 -34.013624   
0.000876    3     -33.973919  1.168927e-10 -33.973919 -34.013624   
0.001229    4     -33.973919  1.667405e-10 -33.973919 -34.013624   
0.001723    5     -33.973919  2.397411e-10 -33.973919 -34.013624   
0.002417    6     -33.973919  3.466170e-10 -33.973919 -34.013624   
0.003391    7     -33.973919  5.030999e-10 -33.973919 -34.013624   
0.004757    8     -33.973919  7.321728e-10 -33.973919 -34.013624   
0.006672    9     -33.973919  1.067444e-09 -33.973919 -34.013624   
0.009359    10    -33.973919  1.558089e-09 -33.973919 -34.013624   
0.013128    11    -33.973919  2.276009e-09 -33.973919 -34.013624   
0.018415    12    -33.973919  3.326359e-09 -33.973919 -34.013624   
0.025830    13    -33.973919  4.862872e-09 -33.973919 -34.013624   
0.036232    14    -33.973919  7.110629e-09 -33.973919 -34.013624   
0.050823    15    -33.973919  1.043642e-08 -33.973919 -34.013624   
0.071291    16    -33.973919  1.605156e-08 -33.973919 -34.013624   
0.100000    17    -33.973919  2.734352e-08 -33.973919 -34.013624   
0.140271    18    -33.973919  5.122416e-08 -33.973919 -34.013624   
0.196760    19    -33.973919  1.012709e-07 -33.973919 -34.013624   
0.275997    20    -33.973919  2.050673e-07 -33.973919 -34.013624   
0.387100    21    -33.973919  4.180737e-07 -33.973919 -34.013624   
0.543100    22    -33.973919  8.513531e-07 -33.973918 -34.013624   
0.761700    23    -33.973919  1.692804e-06 -33.973917 -34.013624   
1.068500    24    -33.973919  3.007065e-06 -33.973916 -34.013625   
1.498800    25    -33.973919  4.668908e-06 -33.973914 -34.013625   
2.102400    26    -33.973919  6.597135e-06 -33.973913 -34.013625   
2.949000    27    -33.973920  8.667058e-06 -33.973911 -34.013626   
4.136600    28    -33.973921  1.083249e-05 -33.973910 -34.013627   
5.802500    29    -33.973923  1.308661e-05 -33.973909 -34.013628   
8.139200    30    -33.973925  1.541952e-05 -33.973909 -34.013629   
11.417000   31    -33.973927  1.781527e-05 -33.973909 -34.013631   
16.014700   32    -33.973931  2.026139e-05 -33.973911 -34.013634   
22.464000   33    -33.973935  2.274586e-05 -33.973913 -34.013637   
31.510500   34    -33.973942  2.525881e-05 -33.973916 -34.013642   
44.200100   35    -33.973949  2.775312e-05 -33.973921 -34.013647   
62.000000   36    -33.973959  2.992945e-05 -33.973928 -34.013654   
85.775000   37    -33.973967  3.150287e-05 -33.973937 -34.013662   
109.550000  38    -33.973976  3.259078e-05 -33.973943 -34.013669   
133.325000  39    -33.973985  3.401840e-05 -33.973951 -34.013677   
157.100000  40    -33.973997  3.664120e-05 -33.973961 -34.013688   
180.875000  41    -33.974015  4.147665e-05 -33.973974 -34.013702   
204.650000  42    -33.974047  5.254247e-05 -33.973995 -34.013728   
228.425000  43    -33.974134  8.960008e-05 -33.974044 -34.013802   
252.200000  44    -33.974334  1.935143e-04 -33.974141 -34.013975   
275.975000  45    -33.974744  4.469162e-04 -33.974298 -34.014334   
299.750000  46    -33.975505  9.986421e-04 -33.974507 -34.015004   
323.525000  47    -33.976806  2.091519e-03 -33.974714 -34.016161   
347.300000  48    -33.978885  4.090120e-03 -33.974795 -34.018013   
371.075000  49    -33.981988  7.458906e-03 -33.974529 -34.020790   
394.850000  50    -33.986149  1.250869e-02 -33.973640 -34.024548   
418.625000  51    -33.991574  1.981088e-02 -33.971763 -34.029477   
442.400000  52    -33.998472  3.003798e-02 -33.968434 -34.035752   
466.175000  53    -34.007053  4.396163e-02 -33.963092 -34.043599   
489.950000  54    -34.017520  6.245282e-02 -33.955067 -34.053242   
513.725000  55    -34.030043  8.643811e-02 -33.943606 -34.064881   
537.500000  56    -34.044791  1.169658e-01 -33.927825 -34.078591   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')